# Introduction to Libraries for Statistical Tests and Regression in Python
- SciPy: Offers foundational statistical functions (t-tests, chi-squared tests, etc.) and simple descriptive statistics.
- Scikit-Learn: Primarily for machine learning; provides straightforward implementations for linear regression and more complex models, but with minimal statistical summaries.
- Statsmodels: Ideal for regression models with in-depth statistical details (e.g., p-values, confidence intervals, ANOVA).

## Basic Statistical Tests with `scipy.stats`

### T-Tests
- One-sample t-test: Compares the mean of a single sample to a known population mean. `ttest_1samp()`
- Two-sample t-test: Compares the means of two independent groups. `ttest_ind()`
- Paired t-test: Compares the means of two related groups (e.g., pre-test and post-test measurements on the same subjects). `ttest_rel()`

In [1]:
import pandas as pd
from scipy import stats

# Example Data
data = pd.DataFrame({
    'sample': [10, 12, 15, 18, 20],
    'group1': [15, 20, 22, 24, 25],
    'group2': [30, 32, 35, 40, 45],
    'pre_test': [80, 85, 90, 95, 100],
    'post_test': [82, 87, 91, 97, 102]
})


- `ttest_1samp`
  - Parameters: `a`: The sample data (as a list, array, or pandas Series).
`popmean`: The population mean to test against.
  - Returns: `t-statistic`: The calculated t-statistic.
`p-value`: The probability of observing the data if the null hypothesis is true.

In [ ]:
# Define population mean
population_mean = 18

# Perform one-sample t-test
t_stat, p_value = stats.ttest_1samp(data['sample'], population_mean)
print(f"One-sample t-test: t-statistic = {t_stat}, p-value = {p_value}")

One-sample t-test: t-statistic = -1.6269784336399213, p-value = 0.1790703787475794


In [ ]:
stats.ttest_1samp(data['sample'], 17)

TtestResult(statistic=-1.0846522890932808, pvalue=0.33907706881839633, df=4)

- `ttest_ind`
  - Parameters:
`a`: The data from the first group.
`b`: The data from the second group.
`equal_var` (optional): A boolean to specify if the variance between groups is assumed to be equal (default is `True`). If `False`, Welch’s t-test is performed, which is more reliable when the two groups have unequal variances.
  - Returns:
  `t-statistic`: The calculated t-statistic.
`p-value`: The probability of observing the data if the null hypothesis is true.

In [ ]:
# Perform two-sample t-test between group1 and group2
t_stat, p_value = stats.ttest_ind(data['group1'], data['group2'])
print(f"Two-sample t-test: t-statistic = {t_stat}, p-value = {p_value}")



Two-sample t-test: t-statistic = -4.668642887938028, p-value = 0.001605340674942487


- `ttest_rel`
  - Parameters:
`a`: The data from the first measurement (e.g., pre-test scores).
`b`: The data from the second measurement (e.g., post-test scores).
  - Returns:
`t-statistic`: The calculated t-statistic.
`p-value`: The probability of observing the data if the null hypothesis is true.


In [ ]:
# Perform paired t-test between pre_test and post_test
t_stat, p_value = stats.ttest_rel(data['pre_test'], data['post_test'])
print(f"Paired t-test: t-statistic = {t_stat}, p-value = {p_value}")



Paired t-test: t-statistic = -9.0, p-value = 0.0008438325176012782


## Overview of ANOVA
ANOVA tests for significant differences between the means of three or more independent groups. It’s commonly used when comparing multiple group means to see if at least one is significantly different.

 `f_oneway()`: One-way ANOVA for comparing multiple independent groups.
  - Parameters: Each group of data is passed as a separate argument. For instance, if there are three groups, they are passed as `f_oneway(group1, group2, group3)`.
  - Returns:
  `F-statistic`: The calculated F-statistic, indicating the ratio of between-group variance to within-group variance.
`p-value`: The probability of observing the data if the null hypothesis is true.

In [ ]:
# Sample data for ANOVA in DataFrame format
anova_data = pd.DataFrame({
    'group1': [23, 25, 27, 29, 31],
    'group2': [22, 24, 26, 28, 30],
    'group3': [30, 32, 34, 36, 38]
})
# Perform one-way ANOVA on group1, group2, and group3
f_stat, p_value = stats.f_oneway(anova_data['group1'], anova_data['group2'], anova_data['group3'])
print(f"One-way ANOVA: F-statistic = {f_stat}, p-value = {p_value}")



One-way ANOVA: F-statistic = 9.5, p-value = 0.003364475059568795


## Statmodels

The statsmodels library in Python is powerful for statistical analysis and modeling, offering several essential functions for linear and logistic regression, hypothesis testing, and ANOVA. Unlike `scikit-learn`, which focuses more on machine learning, statsmodels provides a range of built-in statistical tests and easy-to-interpret summary outputs, ideal for examining and interpreting data.

### Overview of Key statsmodels Functions
- `sm.OLS()`: Ordinary Least Squares regression for linear regression. Good for modeling relationships between continuous variables.
- `smf.ols()`: Formula-based Ordinary Least Squares regression. This function allows us to use R-style formulas to specify models, making it easy to include multiple predictors and interactions.
- `smf.logit()`: Logistic regression, useful when the outcome variable is binary (e.g., yes/no, 0/1).
- `sm.stats.anova_lm()`: ANOVA table for comparing means across multiple groups, often used to assess the impact of categorical variables.
- `summary()`: Available on all fitted models, it provides a summary of coefficients, R-squared values, p-values, and more, making it easy to interpret model results.

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

diamonds = pd.read_csv("https://raw.githubusercontent.com/tidyverse/ggplot2/main/data-raw/diamonds.csv")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Simple Linear Regression
Let's start with a basic linear regression model using statsmodels. We will explore how variables like GDP and literacy rate might relate to life expectancy.

- `sm.add_constant()`: Adds an intercept term to the model, which helps account for the baseline value of the response when all predictors are zero.

In [ ]:
# Define the predictor (carat) and response (price)
X = diamonds['carat']
y = diamonds['price']

# Add a constant (intercept) to the predictor
X = sm.add_constant(X)

# Fit the model
model_ols = sm.OLS(y, X).fit()

# Output the summary
print(model_ols.summary())



                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                 3.041e+05
Date:                Wed, 13 Nov 2024   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:            -4.7273e+05
No. Observations:               53940   AIC:                         9.455e+05
Df Residuals:                   53938   BIC:                         9.455e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2256.3606     13.055   -172.830      0.0

### Multiple Linear Regression

`smf.ols()`- : The formula API allows us to specify the model directly in a string using an R-like syntax.
  - `price ~ carat + C(cut) + C(color) + C(clarity)`: `~` indicates that price is the response variable, while the right side includes predictors (carat, and the categorical variables cut, color, and clarity).
  - ` C()`: Wraps categorical variables, allowing statsmodels to interpret them as factors.

In [ ]:
# Formula-based approach for multiple regression
model_multi = smf.ols('price ~ carat + C(cut) + C(color) + C(clarity)', data=diamonds).fit()
print(model_multi.summary())



                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.916
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                 3.264e+04
Date:                Wed, 13 Nov 2024   Prob (F-statistic):               0.00
Time:                        17:16:12   Log-Likelihood:            -4.5699e+05
No. Observations:               53940   AIC:                         9.140e+05
Df Residuals:                   53921   BIC:                         9.142e+05
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -7362.8022    

### Logistic Regression



In [ ]:
# Create a binary outcome (1 if price is above the median, else 0)
diamonds['high_price'] = (diamonds['price'] > diamonds['price'].median()).astype(int)

# Logistic regression model
model_logit = smf.logit('high_price ~ carat + C(cut) + C(color) + C(clarity)', data=diamonds).fit()
print(model_logit.summary())


Optimization terminated successfully.
         Current function value: 0.051676
         Iterations 12
                           Logit Regression Results                           
Dep. Variable:             high_price   No. Observations:                53940
Model:                          Logit   Df Residuals:                    53921
Method:                           MLE   Df Model:                           18
Date:                Wed, 13 Nov 2024   Pseudo R-squ.:                  0.9254
Time:                        17:17:01   Log-Likelihood:                -2787.4
converged:                       True   LL-Null:                       -37388.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -48.6265      0.975    -49.871      0.000     -50.538     -46.715
C

### ANOVA

In `statsmodels.stats.anova_lm()`, the `typ` parameter specifies the type of sums of squares to calculate for the ANOVA table. There are three common types:

- Type I (`typ=1`): Also known as sequential sums of squares, this method calculates each factor's effect in the order they appear in the model. This type only tests each factor sequentially, meaning the results can depend on the order in which variables are entered.

- Type II (`typ=2`): Also known as "hierarchical" or "partial" sums of squares, this type tests each factor after accounting for all other factors in the model, but without considering interactions. It evaluates the effect of each factor assuming that all other main effects are present. Type II sums of squares are common when factors are balanced (equal sample sizes across groups) or when there are no significant interactions.

- Type III (`typ=3`): This type calculates the "marginal" sums of squares, meaning it tests each factor’s effect while controlling for all other factors, including interactions. This method is most appropriate when interactions are present, and is commonly used in unbalanced designs (unequal sample sizes across groups). However, it can lead to less straightforward interpretations if the model is highly complex.

In [ ]:
# Fit an ANOVA model to assess if cut affects price
anova_model = smf.ols('price ~ C(cut)', data=diamonds).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)
print(anova_table)


                sum_sq       df           F         PR(>F)
C(cut)    1.104175e+10      4.0  175.688717  8.428307e-150
Residual  8.474314e+11  53935.0         NaN            NaN


## Scikit-Learn
Scikit-learn, unlike scipy.stats or statsmodels, is primarily a machine learning library but has some statistical functions, mainly focused on model evaluation, feature selection, and cross-validation. It provides robust tools for creating and evaluating regression models, rather than explicit statistical hypothesis tests.

**scikit-learn's features:**

- Perform regression modeling using `LinearRegression`
- Evaluate model performance with metrics and cross-validation
- Conduct feature selection with various statistical techniques (like ANOVA in `f_regression`)

**Key Functions for Modeling in scikit-learn**
- `LinearRegression()`: Performs linear regression modeling.
- `metrics`: Contains functions for evaluating model performance (e.g., `mean_squared_error`, `r2_score`).
- `cross_val_score()`: Automates cross-validation for model reliability and accuracy assessment.
- `f_regression()`: Conducts ANOVA (Analysis of Variance) F-test for feature selection in regression.


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import f_regression
import seaborn as sns
import numpy as np

In [4]:
# Preprocessing: select relevant features and apply log transformation
diamonds_filtered = diamonds[diamonds['carat'] <= 2.5]  # Focus on diamonds <= 2.5 carats
diamonds_filtered['log_price'] = np.log2(diamonds_filtered['price'])
diamonds_filtered['log_carat'] = np.log2(diamonds_filtered['carat'])

# Define features and target variable
X = diamonds_filtered[['log_carat']]
y = diamonds_filtered['log_price']

<ipython-input-4-0c479e41552c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds_filtered['log_price'] = np.log2(diamonds_filtered['price'])
<ipython-input-4-0c479e41552c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds_filtered['log_carat'] = np.log2(diamonds_filtered['carat'])


We use `LinearRegression` from scikit-learn to fit a linear regression model, which finds the best-fit line by minimizing the difference between observed and predicted values.

- **Define Features and Target**: We set X as the predictor variable (`log_carat`) and y as the target (`log_price`).
- **Initialize Model**: `LinearRegression()` initializes an instance of a linear regression model, which will learn from the data.
- **Fit the Model**: `model.fit(X, y)` estimates the best-fit line by finding values for the intercept and slope(s) that minimize error.
- **View Coefficients**: The intercept `(model.intercept_)` and coefficient `(model.coef_)` help interpret the model:
  - Intercept: Expected log price when `log_carat` is zero.
  - Coefficient: For each 1-unit increase in `log_carat`, the `log_price` increases by the coefficient value.

In [5]:
# Initialize linear regression model
model = LinearRegression()

# Fit model to data
model.fit(X, y)

# View coefficients
print("Intercept:", model.intercept_)
print("Coefficient for log_carat:", model.coef_[0])

Intercept: 12.193863000160825
Coefficient for log_carat: 1.6813710552805718


**Model Evaluation with metrics**

We use metrics such as Mean Squared Error (MSE) and R-squared (R²) to assess how well the model fits the data.

- Predictions: `model.predict(X)` generates predictions for log_price based on log_carat.
- Mean Squared Error (MSE): `mean_squared_error(y, y_pred)` calculates the average squared difference between actual and predicted values. Lower MSE values indicate better model accuracy.
- R-squared (R²): `r2_score(y, y_pred)` measures how well the model explains the variance in log_price. R² values range from 0 to 1, with values closer to 1 indicating a better fit.

In [6]:
# Make predictions
y_pred = model.predict(X)

# Calculate performance metrics
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 0.14192617966013893
R-squared: 0.9334026253208662


**Cross-Validation with `cross_val_score`**

Cross-validation is a method to ensure that the model generalizes well to new data by splitting the dataset into multiple training and testing subsets.

- `cross_val_score`: This function automates cross-validation by splitting the data into 5 “folds” (i.e., subsets), training the model on 4 of them, and testing on the remaining one, then repeating this process to get 5 R² scores.
- Average R-squared: By averaging the R² scores across folds, we get an estimate of the model's generalization ability on new data. Higher average scores mean the model is more reliable.

In [ ]:
# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

print("Cross-Validation R-squared Scores:", cv_scores)
print("Average Cross-Validation R-squared:", cv_scores.mean())


Cross-Validation R-squared Scores: [ 0.84531724  0.86304132  0.95322791 -0.02249409  0.7341177 ]
Average Cross-Validation R-squared: 0.6746420147937064


**Feature Selection with `f_regression`**

`f_regression` is useful to determine if additional features significantly improve the model. It calculates the F-statistic and corresponding p-value for each feature, indicating if the feature is relevant to the target variable.

- OneHotEncoder and ColumnTransformer: Many machine learning models require categorical variables to be numeric. OneHotEncoder transforms each category into a new binary column (e.g., cut_Premium, cut_Good).
- F-statistics and p-values: The F-statistic measures how much each feature explains the variance in y when included in the model. Features with high F-statistics and low p-values are likely important in explaining log_price.-

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Encode categorical features for regression
X_full = diamonds_filtered[['log_carat', 'color', 'cut', 'clarity']]
y = diamonds_filtered['log_price']

# Transform categorical features to numerical values
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['log_carat']),
        ('cat', OneHotEncoder(), ['color', 'cut', 'clarity'])
    ])

# Fit preprocessing on full feature set
X_full_transformed = preprocessor.fit_transform(X_full)

# Feature selection test
f_stat, p_values = f_regression(X_full_transformed, y)

# Output the F-statistic and p-values
print("F-statistics:", f_stat)
print("p-values:", p_values)


F-statistics: [7.54207840e+05 2.11202284e+02 4.91361014e+02 4.89640836e+00
 3.76023413e-01 1.58963538e+02 3.12772356e+02 3.43459215e+02
 1.36981712e+02 1.53209420e+01 7.48910179e+02 4.81275497e+02
 2.71026446e+00 2.29912815e+01 2.58640553e+02 6.91246416e+01
 1.51814834e+03 3.18847804e+01 6.06647487e+00 8.20957093e+02
 3.48070759e+02]
p-values: [0.00000000e+000 9.25078791e-048 2.20714234e-108 2.69167948e-002
 5.39741751e-001 2.14600310e-036 8.57370356e-070 1.94943118e-076
 1.32934753e-031 9.08169770e-005 9.21736841e-164 3.30143297e-106
 9.97103009e-002 1.63172068e-006 4.63900339e-058 9.45665585e-017
 0.00000000e+000 1.64416238e-008 1.37802729e-002 3.35066495e-179
 1.95909702e-077]
